In [1]:
import pandas as pd
from pathlib import Path

In [2]:
Path.cwd()

PosixPath('/Users/zhuoran/Projects/trigger_analyzer/ana')

In [3]:
file_name = Path.cwd().parent.joinpath("data", "stas","data_metrics.csv")

In [4]:
df = pd.read_csv(file_name)
df.head()

,pkg_type,location,function,attack_type,trigger_mechanism,hide
0,pypi,setup,spyware and information stealing,dropper/malware,execute automatically upon download,base64-encoded
1,pypi,variable names in Chinese,"access the clipboard, replay crypto wallet add...",typosquatting,installation,obfuscation
2,pypi,setup,spyware and information stealing,dropper/malware,execute automatically upon download,base64-encoded
3,pypi,setup,spyware and information stealing,dropper/malware,execute automatically upon download,base64-encoded
4,pypi,setup,spyware and information stealing,dropper/malware,execute automatically upon download,base64-encoded


In [5]:
df["pkg_type"].value_counts()

pkg_type
npm     9766
pypi    6487
ruby      17
rust       2
Name: count, dtype: int64

In [6]:
df["location"].value_counts()

location
entrypoint/download                                9179
setup                                              5462
variable names in Chinese                           900
socket or other communication protocols modules     594
three inter hooked files                             42
functions                                            35
requests function                                    10
specific function                                     1
Name: count, dtype: int64

In [7]:
df["function"].value_counts()

function
install powerful malware                                                 9179
spyware and information stealing                                         5425
access the clipboard, replay crypto wallet address                        900
build communication tunnel with C2                                        594
steal source code and secrets                                              42
gain persistence/exfiltrate host information                               27
steal sensitive credentials                                                21
multi-stage malware download                                               14
information steal/local passwords                                          10
download an open source cryptominer/subprocess/steal host information      10
download payload                                                            1
information gathering                                                       1
Name: count, dtype: int64

In [8]:
df["attack_type"].value_counts()

attack_type
data exfiltration/root shell        9179
dropper/malware                     5425
typosquatting                        937
C2 communication                     594
execute malicious commands            48
typosquatting/starjacking             31
social engineering/typosquatting      14
typosquatting/malware                  1
Name: count, dtype: int64

In [9]:
df["trigger_mechanism"].value_counts()

trigger_mechanism
download or upon installation                        9179
execute automatically upon download                  5462
installation                                          900
running network connection                            577
execute upon installation                              42
functions called/send message                          21
network connection                                     17
execute upon installation/asynchronous operations      14
launch upon usage                                      10
function call/condition/variable configuration          1
Name: count, dtype: int64

In [10]:
df["hide"].value_counts()

hide
base64-encoded                          5425
obfuscation                              900
split payloads                            42
steganography/obfuscation                 27
base64-encoded/embedded in functions      21
split payloads/encoding                   14
Name: count, dtype: int64